In [1]:
import pandas as pd
import numpy as np

In [124]:
# First off merge the generator data from powerworld with generator data from excel
powerworld_gen = pd.read_csv('powerworldGen.csv')
gen_units = pd.read_csv('GenUnits.csv')
curves = pd.read_csv("pw_curves.csv")

#discard various unused columns
genu_drop = ["GenFuelType", "Utility ID", "Utility Name", "Plant Name", "State", "County", "Prime Mover", "EIA Generator ID",
            "EIA-860 Plant Name", "Prime Mover", "Unit Code", "Ownership", "Duct Burners", "Can Bypass Heat Recovery Steam Generator?",
            "RTO/ISO LMP Node Designation", "RTO/ISO Location Designation for Reporting Wholesale Sales Data to FERC", "Nameplate Capacity (MW)",
            "Nameplate Power Factor", "Summer Capacity (MW)", "Winter Capacity (MW)", "Uprate or Derate Completed During Year",
            "Month Uprate or Derate Completed", "Year Uprate or Derate Completed", "Status", "Synchronized to Transmission Grid",
            "Operating Month", "Operating Year", "Planned Retirement Month", "Planned Retirement Year", "Associated with Combined Heat and Power System",
            "Sector Name", "Sector", "Topping or Bottoming", "Energy Source 2", "Energy Source 3", "Energy Source 4", "Energy Source 5", "Energy Source 6",
            "Startup Source 1", "Startup Source 2", "Startup Source 3", "Startup Source 4", "Solid Fuel Gasification System?", "Carbon Capture Technology?",
            "Turbines or Hydrokinetic Buoys", "Fluidized Bed Technology?", "Pulverized Coal Technology?","Stoker Technology?","Other Combustion Technology?",
            "Subcritical Technology?", "Supercritical Technology?","Ultrasupercritical Technology?","Planned Net Summer Capacity Uprate (MW)",
            "Planned Net Winter Capacity Uprate (MW)","Planned Uprate Month","Planned Uprate Year","Planned Net Summer Capacity Derate (MW)","Planned Net Winter Capacity Derate (MW)",
             "Planned Derate Month", "Planned Derate Year","Planned New Prime Mover","Planned Energy Source 1","Planned New Nameplate Capacity (MW)",
             "Planned Repower Month","Planned Repower Year","Other Planned Modifications?","Other Modifications Month","Other Modifications Year","Multiple Fuels?",
             "Cofire Fuels?","Switch Between Oil and Natural Gas?"]
gen_units = gen_units.drop(genu_drop, axis=1)

pw_drop = ["Status","AGC","# Cost Curve Points","IOB","IOC","Variable O&M","IOD","AVR","Enforce MW Limits","Cost Model"]
powerworld_gen = powerworld_gen.drop(pw_drop, axis = 1)


# create unique IDs to merge the data on
powerworld_gen["BUS UID"] = powerworld_gen.apply(lambda x: "{}{}".format(x["Number of Bus"], x["ID"]), axis = 1)
gen_units["BUS UID"] = gen_units.apply(lambda x: "{}{}".format(x["BusNum"], x["GenID"]), axis = 1)
curves["BUS UID"] = curves.apply(lambda x: "{}{}".format(int(x["Number of Bus"]), int(x["ID"])), axis = 1)

curves_drop = ["Name of Bus", "Area Name of Gen", "Status", "AGC", "Fuel Type","Unit Type", "Cost Shift $/MWh", "Cost Multiplier",
              "Fixed Cost(Mbtu/hr)", "MW Break 6", "MW Break 7", "MW Break 8", "MW Break 9", "MW Break10", "MW Break11", "MW Break12", 
               "MW Break13", "MW Break14", "MW Break15", "MW Break16", "MW Break17", "MW Break18", "MW Break19", "MW Break20",
              "MWh Price 6", "MWh Price 7","MWh Price 8","MWh Price 9","MWh Price10","MWh Price11","MWh Price12","MWh Price13","MWh Price14",
               "MWh Price15","MWh Price16","MWh Price17","MWh Price18","MWh Price19","MWh Price20", "Number of Bus", "ID",
              "Gen MW", "Max MW", "Min MW", "Fuel Cost"]
curves = curves.drop(curves_drop, axis = 1)

gen_merge = powerworld_gen.merge(gen_units, how = "left", on = "BUS UID", validate="one_to_one", copy=False)
gen_merge_1 = gen_merge.merge(curves, how = "left", on = "BUS UID", validate="one_to_one", copy=False)
gen_merge_1.to_csv("gen_merge.csv")


In [125]:
# add in the logic to calculate cold start
def cold_start(x):
    if(x == "OVER"):
        return 13
    elif(x == "12H"):
        return 12
    elif(x == "1H"):
        return 1
    elif(x == "10M"):
        return 1/6
    else:
        return 0

gen_master = pd.read_csv("gen_merge.csv")

gen_master["Start Time Cold Hr"] = gen_master["Time from Cold Shutdown to Full Load"].apply(cold_start)
gen_master["Start Time Warm Hr"] = gen_master["Start Time Cold Hr"].apply(lambda x: x/2)
gen_master["Start Time Hot Hr"] = gen_master["Start Time Cold Hr"].apply(lambda x: x/4)

gen_master = gen_master.drop("Time from Cold Shutdown to Full Load", axis = 1)
gen_master.to_csv("gen_step_2.csv")

In [135]:
# merge in UC parameters
gen_master = pd.read_csv("gen_step_2.csv")
uc_param = pd.read_csv("GenUCParams.csv")

# create unique ids
uc_param["GEN UID"] = uc_param.apply(lambda x: "{}_{}_{}".format(x["PlantCode"], x["Technology"].replace(" ",""), x["GeneratorID"]), axis=1)
gen_master["GEN UID"] = gen_master.apply(lambda x: "{}_{}_{}".format(x["Plant Code"], x["Technology"].replace(" ",""), x["Generator ID"]), axis=1)


# drop extraneous
uc_drop = ["PlantName", "WinterCapacityMW", "PlantCode","GeneratorID", "Technology", "WinterCapacityMW"]
uc_param = uc_param.drop(uc_drop, axis = 1)

gen_step_3 = gen_master.merge(uc_param, how = "left", on = "GEN UID", validate="one_to_one", copy=False)
gen_step_3.to_csv("gen_step_3.csv")

In [147]:
# rename all the relevant columns
gen_master = pd.read_csv("gen_step_3.csv")

gen_master["Non Fuel Start Cost $"] = np.zeros(731)
gen_master = gen_master.rename({"Number of Bus":"Bus ID",
                               "Technology": "Unit Type",
                               "Fuel Type": "Fuel",
                               "Min MW": "PMin MW",
                               "Max MW": "PMax MW",
                               "minDnHr": "Min Down Time Hr",
                               "minUpHr": "Min Up Time Hr",
                               "RRMWperMin": "Ramp Rate MW/Min",
                               "Fuel Cost":"Fuel Price $/MMBTU"}, axis=1)
# artifacts of the merges
gen_master = gen_master.drop(["Unnamed: 0","Unnamed: 0.1", "Unnamed: 0.1.1"], axis=1)

gen_master.to_csv("gen_step_4.csv")

In [35]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import interp1d

# read in csv
df = pd.read_csv("gen_step_4.csv")

# add information for start heats, unit group based on dictionary lookup
start_heat = {
    "Solar Photovoltaic": [0,0,0],
    "Petroleum Coke": [68,44,38],
    "Other Gases": [68,44,38],
    "Onshore Wind Turbine": [0,0,0],
    "Nuclear": [78978, 0, 9999],
    "Natural Gas Steam Turbine":  [1457.4, 1122.5,452.8],
    "Natural Gas Fired Combustion Turbine": [1457.4, 1122.5,452.8],
    "Natural Gas Internal Combustion Engine": [7215.1, 4536.1, 3196.6],
    "Natural Gas Fired Combined Cycle": [7215.1, 4536.1, 3196.6],
    "Conventional Steam Coal": [45437, 36480, 27540],
    "Conventional Hydroelectric": [0,0,0],
    "Batteries": [0,0,0],
    "All Other": [0,0,0],
    "Wood/Wood Waste Biomass": [0,0,0],
    "Other Waste Biomass": [0,0,0]
}

output_pct_dict = {
    "Solar Photovoltaic": [0,0,0,0],
    "Petroleum Coke": [0.4,0.6,0.8,1.0],
    "Other Gases": [0.1,0.5,0.8,1.0],
    "Onshore Wind Turbine": [0,0,0,0],
    "Nuclear": [0.99,0.993, 0.996, 1],
    "Natural Gas Steam Turbine": [0.4,0.6,0.8,1],
    "Natural Gas Fired Combustion Turbine": [0.4,0.6,0.8,1],
    "Natural Gas Internal Combustion Engine": [0.478, 0.652, 0.827, 1], 
    "Natural Gas Fired Combined Cycle":[0.478, 0.652, 0.827, 1],
    "Conventional Steam Coal": [0.25,0.5,0.75,1],
    "Conventional Hydroelectric": [1,0,0,0],
    "All Other": [0.4,0.6,0.8,1],
    "Batteries": [1,0,0,0],
    "Wood/Wood Waste Biomass": [0,0,0,0],
    "Other Waste Biomass": [0,0,0,0]
}

df.dropna(subset=["Unit Type"], inplace=True)

df["Start Heat Cold MBTU"] = df["Unit Type"].apply(lambda x: start_heat.get(x)[0])
df["Start Heat Warm MBTU"] = df["Unit Type"].apply(lambda x: start_heat.get(x)[1])
df["Start Heat Hot MBTU"] = df["Unit Type"].apply(lambda x: start_heat.get(x)[2])

df["Output_pct_0"] = df["Unit Type"].apply(lambda x: output_pct_dict.get(x)[0])
df["Output_pct_1"] = df["Unit Type"].apply(lambda x: output_pct_dict.get(x)[1])
df["Output_pct_2"] = df["Unit Type"].apply(lambda x: output_pct_dict.get(x)[2])
df["Output_pct_3"] = df["Unit Type"].apply(lambda x: output_pct_dict.get(x)[3])

def create_unit_group(x):
    if(x["Unit Type"] in ["Solar Photovoltaic", "Wind", "Batteries", "Onshore Wind Turbine"]):
        return x["Unit Type"]
    else:
        return "U{}".format(int(x["PMax MW"]))   
    

df["Unit Group"] = df.apply(create_unit_group, axis=1)

# function based on Ronnie's script that can be applied elementwise
def apply_interpolation(x):
    if(x["Unit Type"] in ["Solar Photovoltaic", "Wind", "Batteries", "Onshore Wind Turbine"]):
        """
        x["HR_avg_0"] = 0
        x["HR_incr_1"] = 0
        x["HR_incr_2"] = 0
        x["HR_incr_3"] = 0
        """
        return pd.Series([x["GEN UID"],0,0,0,0], index = ["GEN UID","HR_avg_0", "HR_incr_1", "HR_incr_2", "HR_incr_3"])
    # Variable Definitions
    fuel_cost = x["Fuel Price $/MMBTU"] # fuel cost for bus number 111184 (cell D2 of powerworld_curves.csv provided by ER)
    # units: $/BTU
    #MWh breaks in heat cost curve for bus 111184 (cells E,G,I,K,M2 of powerworld_curves.csv)
    mw = np.array([0, x["MW Break 1"], x["MW Break 2"], x["MW Break 3"], x["MW Break 4"], x["MW Break 5"], x["PMax MW"]]) # units: MWh
    # MWh price in heat cost curve for bus 111184 (cells F,H,J,L,N2 in powerworld_curves.csv)
    mwh_price = np.array([x["MWh Price 1"],x["MWh Price 1"], x["MWh Price 2"], x["MWh Price 3"], x["MWh Price 4"], x["MWh Price 5"], x["MWh Price 5"]]) # Units: $/MWh
    #plt.figure(1)
    #plt.plot(mw, mwh_price, 'x')
    #plt.xlabel("MW")
    #plt.ylabel("$/MWh")
    BTU_per_kWh = (mwh_price / fuel_cost) * 1000
    output_pct = np.array([x["Output_pct_0"], x["Output_pct_1"], x["Output_pct_2"], x["Output_pct_3"]])
    PMax = x["PMax MW"]
    interpolated = interp1d(mw, BTU_per_kWh, kind='linear')
    HR = interpolated(output_pct * PMax)
    return pd.Series([x["GEN UID"], HR[0], HR[1]-HR[0], HR[2]-HR[1], HR[3]-HR[2]], index = ["GEN UID", "HR_avg_0", "HR_incr_1", "HR_incr_2", "HR_incr_3"])
    # plt.figure(2)
    # plt.plot(output_pct, HR_average)
hr = df.apply(apply_interpolation, axis=1)

df = df.merge(hr, on="GEN UID", validate="one_to_one")

df.dropna(subset=["Ramp Rate MW/Min"], inplace=True)

df.to_csv("./gen_step_5.csv")

In [37]:
df = pd.read_csv("./gen_step_5.csv")

df["Name of Bus"] = df["Name of Bus"].str.replace(" ", "_")
df["Name of Bus"] = [x[:-2] for x in df["Name of Bus"]]

df = df[df["Fuel"] != "WH (Waste Heat)"]
df = df[df["Fuel"] != "WDS (Wood/Wood Waste Solids)"]
df = df[df["Unit Type"] != "Other Gases"]

df.to_csv("./TX_data/SourceData/gen.csv")

In [20]:
bus = pd.read_csv("./bus_step_1.csv")

bus["Bus Name"] = bus["Bus Name"].str.replace(" ", "_")
bus["Sub Name"] = bus["Sub Name"].str.replace(" ", "_")

bus.to_csv("./TX_data/SourceData/bus.csv")

branch = pd.read_csv("./branch_step_1.csv")

branch["From Name"] = branch["From Name"].str.replace(" ", "_")
branch["To Name"] = branch["To Name"].str.replace(" ", "_")

branch.to_csv("./TX_data/SourceData/branch.csv")

In [6]:
# match the UIDs to the NREL data
wind_map = pd.read_csv("./NREL_data/Texas7k_NREL_wind_map.csv")
wind_map["GEN UID"] = wind_map.apply(lambda x: "{}_OnshoreWindTurbine_{}".format(x["EIA-860 Plant Code"], x["Texas7k GenID"]), axis=1)
wind_map.to_csv("./NREL_data/Texas7k_NREL_wind_map.csv")